In [ ]:
#import pytensor
#print(pytensor.config.cxx)

#set up g++ and openBLAS

#import pytensor
#print(dir(pytensor.config))

#import pytensor
#pytensor.config.blas__ldflags = '-LC:\\OpenBLAS\\lib -lopenblas'
#print(pytensor.config.blas__ldflags)

#import pytensor
#print("BLAS flags:", pytensor.config.blas__ldflags)
# print("Computation Mode:", pytensor.config.mode)


"C:\mingw64\bin\g++.EXE"
['DebugMode__check_c', 'DebugMode__check_finite', 'DebugMode__check_preallocated_output', 'DebugMode__check_preallocated_output_ndim', 'DebugMode__check_py', 'DebugMode__check_strides', 'DebugMode__patience', 'DebugMode__warn_input_not_reused', 'NanGuardMode__action', 'NanGuardMode__big_is_error', 'NanGuardMode__inf_is_error', 'NanGuardMode__nan_is_error', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_config_var_dict', '_flags_dict', '_pytensor_cfg', '_pytensor_raw_cfg', 'add', 'allow_gc', 'base_compiledir', 'blas__check_openmp', 'blas__ldflags', 'cast_policy', 'change_flags', 'check_input', 'check_stack_trace', 'cmodule__age_thresh_use', 'cmodule_

In [4]:
# pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


# USE ADVI (only 1000 rows for testing):

In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score



WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# Load dataset
df = pd.read_excel("book_ratings.xlsx")


In [3]:

# Select relevant columns
df = df[['User-ID', 'ISBN', 'Book-Rating']]

In [4]:
# Drop zero values as they are not the actual rating
df = df[df['Book-Rating'] > 0]

In [5]:
# Check the size of the data
len(df)

383840

In [6]:
df.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276744,038550120X,7
13,276747,0060517794,9


In [7]:
# Create a dataframe grouped by unique users
def create_df_user_reviews(df):
    # Count the number of reviews per user
    user_counts = df['User-ID'].value_counts()
    
    # Store number of reviews per user in a new DataFrame
    df_user_reviews = user_counts.reset_index()
    df_user_reviews.columns = ['User-ID', 'Review-Count']
    
    return df_user_reviews

In [8]:
df_user_reviews = create_df_user_reviews(df)
df_user_reviews

,User-ID,Review-Count
0,11676,6943
1,98391,5691
2,189835,1899
3,153662,1845
4,23902,1180
...,...,...
68086,114767,1
68087,114771,1
68088,114772,1
68089,114777,1


In [9]:
# Create a dataframe with the users that made less then or equal to 20 reviews
def create_df_filtered(df):
    # Count the number of reviews per user
    user_counts = df['User-ID'].value_counts()
    
    # Remove users with 20 or more reviews
    valid_users = user_counts[user_counts >= 2].index
    df_filtered = df[df['User-ID'].isin(valid_users)]
    
    return df_filtered

In [10]:
df = create_df_filtered(df)
len(df)

344617

In [11]:
# **Downsample to 1000 random rows for testing**
df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [12]:
# Encode User-ID and ISBN as categorical for indexing
df['User-Index'] = df['User-ID'].astype("category").cat.codes
df['Book-Index'] = df['ISBN'].astype("category").cat.codes

# **Remap indices to contiguous range** (Fixes the IndexError)
df['User-Index'] = df['User-Index'].astype("category").cat.codes
df['Book-Index'] = df['Book-Index'].astype("category").cat.codes

In [13]:
# Compute rating counts per user and book
user_rating_counts = df.groupby('User-Index')['Book-Rating'].count()
book_rating_counts = df.groupby('Book-Index')['Book-Rating'].count()

# Avoid division by zero
user_rating_counts[user_rating_counts == 0] = 1
book_rating_counts[book_rating_counts == 0] = 1

In [14]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [15]:
# Convert to numpy arrays for modeling
train_user_ids = train_df['User-Index'].values
test_user_ids = test_df['User-Index'].values
train_book_ids = train_df['Book-Index'].values
test_book_ids = test_df['Book-Index'].values
train_ratings = train_df['Book-Rating'].values # Using raw ratings for Poisson
test_ratings = test_df['Book-Rating'].values

In [16]:
# Get updated number of unique users and books
num_users = df['User-Index'].nunique()
num_books = df['Book-Index'].nunique()

print("Number of unique users:", num_users)
print("Number of unique books:", num_books)

Number of unique users: 858
Number of unique books: 972


In [ ]:
'''
# Set latent dimension 
#latent_dim = 5
latent_dim = 2

# Bayesian Probabilistic Matrix Factorization Model with Gamma-Poisson
with pm.Model() as model:
    # Prior for global mean rating
    mu = pm.Gamma("mu", alpha=2, beta=0.5)
    
    # ----- User and book bias priors (Adjust sigma for best performance) -----
    user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)

    # Hierarchical priors for latent factors (beta=1 worked best)
    sigma_u = pm.HalfCauchy("sigma_u", beta=1)
    sigma_b = pm.HalfCauchy("sigma_b", beta=1)
    
    user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

    # Expected rating using Poisson lambda
    lambda_rating = pm.math.exp(
        mu +
        user_bias[train_user_ids] +
        book_bias[train_book_ids] +
        (user_factors[train_user_ids] * book_factors[train_book_ids]).sum(axis=1)
    )

    # Poisson likelihood
    ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings)
    
    # Use ADVI for fast variational inference instead of NUTS
    print("Running Variational Inference (ADVI)...")
    approx = pm.fit(n=50000, method="advi")
    trace = approx.sample(draws=2000)

# **Extract posterior values manually since PyMC won't sample `ratings_obs`**
with model:
    print("\nManually Generating Predictions Using Posterior Samples...")
    
    # Extract posterior values
    mu_post = trace.posterior["mu"].mean().item()
    user_bias_post = trace.posterior["user_bias"].mean(dim=("chain", "draw")).values
    book_bias_post = trace.posterior["book_bias"].mean(dim=("chain", "draw")).values
    user_factors_post = trace.posterior["user_factors"].mean(dim=("chain", "draw")).values
    book_factors_post = trace.posterior["book_factors"].mean(dim=("chain", "draw")).values

    # Compute expected ratings
    predicted_ratings = np.exp(
        mu_post + 
        user_bias_post[test_user_ids] + 
        book_bias_post[test_book_ids] +
        (user_factors_post[test_user_ids] * book_factors_post[test_book_ids]).sum(axis=1)
    )

    print("\nExample of Predicted Ratings (posterior predictive mean):")
    print(predicted_ratings[:5])
    
# Evaluation Metrics
mae = mean_absolute_error(test_ratings, predicted_ratings)
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_ratings))

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")'
'''

Running Variational Inference (ADVI)...


Output()

Finished [100%]: Average Loss = 2,783.3



Manually Generating Predictions Using Posterior Samples...

Example of Predicted Ratings (posterior predictive mean):
[5.84627521 5.69960012 5.85902991 5.77918565 5.8529554 ]
Mean Absolute Error (MAE): 2.2448
Root Mean Squared Error (RMSE): 2.5626


In [ ]:
''''
# Evaluation Metrics
mae = mean_absolute_error(test_ratings, predicted_ratings)
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_ratings))

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")'
'''

Mean Absolute Error (MAE): 2.2448
Root Mean Squared Error (RMSE): 2.5626


In [17]:

# Evaluation of Precision, Recall, MAE, and RMSE

def evaluate_predictions(true_ratings, predicted_ratings, threshold=7):
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    
    # Convert to binary relevance (1 if rating >= threshold, else 0)
    true_binary = (true_ratings >= threshold).astype(int)
    predicted_binary = (predicted_ratings >= threshold).astype(int)
    
    precision = precision_score(true_binary, predicted_binary, average='micro')
    recall = recall_score(true_binary, predicted_binary, average='micro')
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")


'''
# Running evaluation
predicted_train_ratings = np.exp(
    trace.posterior["mu"].mean().item() +
    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[train_user_ids] +
    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[train_book_ids]
)
predicted_test_ratings = np.exp(
    trace.posterior["mu"].mean().item() +
    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[test_user_ids] +
    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[test_book_ids]
)

evaluate_predictions(train_ratings, predicted_train_ratings)
evaluate_predictions(test_ratings, predicted_test_ratings)'

'''

'\n# Running evaluation\npredicted_train_ratings = np.exp(\n    trace.posterior["mu"].mean().item() +\n    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[train_user_ids] +\n    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[train_book_ids]\n)\npredicted_test_ratings = np.exp(\n    trace.posterior["mu"].mean().item() +\n    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[test_user_ids] +\n    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[test_book_ids]\n)\n\nevaluate_predictions(train_ratings, predicted_train_ratings)\nevaluate_predictions(test_ratings, predicted_test_ratings)\'\n\n'

In [19]:
!pip install scikit-optimize 

     -------------------------------------- 107.8/107.8 kB 6.1 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl (161 kB)



[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import numpy as np
import pymc as pm
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score
from skopt import gp_minimize
from skopt.space import Real
latent_dim = 2
# Reduce dataset size for speed
subset_size = 500  # Adjust based on hardware
train_idx_subset = np.random.choice(len(train_user_ids), subset_size, replace=False)

train_user_ids_small = train_user_ids[train_idx_subset]
train_book_ids_small = train_book_ids[train_idx_subset]
train_ratings_small = train_ratings[train_idx_subset]

subset_size = min(subset_size // 2, len(test_user_ids))  # Prevent larger sampling
test_idx_subset = np.random.choice(len(test_user_ids), subset_size, replace=False)

test_user_ids_small = test_user_ids[test_idx_subset]
test_book_ids_small = test_book_ids[test_idx_subset]
test_ratings_small = test_ratings[test_idx_subset]

# Define Bayesian optimization function
def objective(params):
    sigma_u_val, sigma_b_val = params

    with pm.Model() as model:
        # Global mean rating
        mu = pm.Gamma("mu", alpha=2, beta=0.5)

        # User and book biases
        user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
        book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)

        # Latent factors with tunable sigma
        sigma_u = pm.HalfCauchy("sigma_u", beta=sigma_u_val)
        sigma_b = pm.HalfCauchy("sigma_b", beta=sigma_b_val)

        user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
        book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

        # Expected ratings
        lambda_rating = pm.math.exp(
            mu +
            user_bias[train_user_ids_small] +
            book_bias[train_book_ids_small] +
            (user_factors[train_user_ids_small] * book_factors[train_book_ids_small]).sum(axis=1)
        )

        # Poisson likelihood
        ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings_small)

        # Variational inference with reduced steps
        approx = pm.fit(n=5000, method="advi")
        trace = approx.sample(draws=1000)

    # Extract posterior values
    with model:
        mu_post = trace.posterior["mu"].mean().item()
        user_bias_post = trace.posterior["user_bias"].mean(dim=("chain", "draw")).values
        book_bias_post = trace.posterior["book_bias"].mean(dim=("chain", "draw")).values
        user_factors_post = trace.posterior["user_factors"].mean(dim=("chain", "draw")).values
        book_factors_post = trace.posterior["book_factors"].mean(dim=("chain", "draw")).values

    # Compute predictions
    predicted_ratings = np.exp(
        mu_post +
        user_bias_post[test_user_ids_small] +
        book_bias_post[test_book_ids_small] +
        (user_factors_post[test_user_ids_small] * book_factors_post[test_book_ids_small]).sum(axis=1)
    )

    # Evaluate precision
    threshold = 7
    true_binary = (test_ratings_small >= threshold).astype(int)
    predicted_binary = (predicted_ratings >= threshold).astype(int)

    precision = precision_score(true_binary, predicted_binary, average='micro')

    return -precision  # Minimize negative precision (maximize precision)

# Define search space
search_space = [Real(0.1, 2.0, name="sigma_u"), Real(0.1, 2.0, name="sigma_b")]

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=10, random_state=42)

# Best found values
best_sigma_u, best_sigma_b = result.x
print(f"Optimal sigma_u: {best_sigma_u}, Optimal sigma_b: {best_sigma_b}")


Finished [100%]: Average Loss = 27,105


Finished [100%]: Average Loss = 15,059


Finished [100%]: Average Loss = 14,756


Finished [100%]: Average Loss = 30,955


Finished [100%]: Average Loss = 82,664


...

Finished [100%]: Average Loss = 27,345


Finished [100%]: Average Loss = 46,308


Finished [100%]: Average Loss = 14,872


Finished [100%]: Average Loss = 24,360


Optimal sigma_u: 1.6134316750344428, Optimal sigma_b: 0.4485261007457112


In [23]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"betas_{timestamp}.txt"
with open(filename, "w") as f:
    f.write(f"Optimal sigma_u: {best_sigma_u}\n")
    f.write(f"Optimal sigma_b: {best_sigma_b}\n")

print(f"Saved parameters to {filename}")


Saved parameters to betas_2025-03-08_00-21-03.txt


In [ ]:
# ---- Bayes General Multi-Step Lookahead Recommendation ---- #

def bayes_general_recommendation(user_index, book_indices, trace, top_k=5, exploration_factor=0.5, regret_threshold=0.8, max_regret=2.0):
    """
    Multi-step lookahead Bayesian regret minimization for recommending 5 books.
    """
    mu_samples = trace.posterior["mu"].values
    user_bias_samples = trace.posterior["user_bias"].values[:, :, user_index]
    book_bias_samples = trace.posterior["book_bias"].values[:, :, book_indices]
    user_factors_samples = trace.posterior["user_factors"].values[:, :, user_index, :]
    book_factors_samples = trace.posterior["book_factors"].values[:, :, book_indices, :]

    num_samples = mu_samples.shape[1]  # Number of posterior samples
    
    # Compute expected rewards using posterior sampling
    expected_rewards = np.mean(
        np.exp(mu_samples[:, :, None] + user_bias_samples[:, :, None] + book_bias_samples +
               np.sum(user_factors_samples[:, :, None, :] * book_factors_samples, axis=-1)), axis=1
    )

    # Compute variance (uncertainty measure)
    rating_uncertainty = np.var(
        np.exp(mu_samples[:, :, None] + user_bias_samples[:, :, None] + book_bias_samples +
               np.sum(user_factors_samples[:, :, None, :] * book_factors_samples, axis=-1)), axis=1
    )
    
    # Compute Bayesian regret
    best_expected_reward = np.max(expected_rewards, axis=1)
    regrets = best_expected_reward[:, None] - expected_rewards

    # Cap regret to prevent extreme exploration
    regrets = np.clip(regrets, 0, max_regret)

    # Apply regret threshold
    should_explore = regrets > regret_threshold

    # Compute future learning potential
    expected_future_gain = exploration_factor * rating_uncertainty

    # Compute exploration-adjusted score
    exploration_score = expected_rewards + expected_future_gain

    # Rank books
    ranked_books = np.argsort(-exploration_score, axis=1)  # Sort in descending order

    # Select top-k books for recommendation
    selected_books = [book_indices[i] for i in ranked_books[0, :top_k]]

    return selected_books

# Example usage: Recommend 5 books for a user
user_id_example = 42  # Replace with an actual user ID
book_pool = np.arange(num_books)  # Assuming all books are available

recommended_books = bayes_general_recommendation(user_id_example, book_pool, trace, top_k=5)
print("\nTop-5 Recommended Books for User", user_id_example, ":", recommended_books)


Top-5 Recommended Books for User 42 : [644, 28, 785, 250, 900]


In [80]:
# Evaluation of Precision, Recall, MAE, and RMSE in the top 5 recommendations

def evaluate_recommendations(user_ids, book_pool, trace, top_k=5):
    total_precision = 0
    total_recall = 0
    total_mae = 0
    total_rmse = 0
    user_count = len(user_ids)
    
    for user in user_ids:
        actual_books = set(test_df[test_df['User-Index'] == user]['Book-Index'].values)
        actual_ratings = test_df[test_df['User-Index'] == user]['Book-Rating'].values
        recommended_books = set(bayes_general_recommendation(user, book_pool, trace, top_k))
        
        if len(actual_books) > 0:
            precision = len(recommended_books & actual_books) / top_k
            recall = len(recommended_books & actual_books) / len(actual_books)
            predicted_ratings = np.array([trace.posterior["mu"].mean().item() + trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[user] + trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[book] for book in actual_books])
            
            mae = mean_absolute_error(actual_ratings, predicted_ratings)
            rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
        else:
            precision = 0
            recall = 0
            mae = 0
            rmse = 0
        
        total_precision += precision
        total_recall += recall
        total_mae += mae
        total_rmse += rmse
    
    avg_precision = total_precision / user_count
    avg_recall = total_recall / user_count
    avg_mae = total_mae / user_count
    avg_rmse = total_rmse / user_count
    
    print(f"Average Precision@{top_k}: {avg_precision:.4f}")
    print(f"Average Recall@{top_k}: {avg_recall:.4f}")
    print(f"Mean Absolute Error (MAE): {avg_mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {avg_rmse:.4f}")

# Running evaluation
book_pool = np.arange(num_books)
evaluate_recommendations(test_user_ids, book_pool, trace, top_k=5)


Average Precision@5: 0.0030
Average Recall@5: 0.0100
Mean Absolute Error (MAE): 6.0586
Root Mean Squared Error (RMSE): 6.0685
